In [23]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

# Please download the inception_v3
#
# !wget --no-check-certificate \
#     https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
#     -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [24]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [25]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras

# Flatten the output later to 1 dimension
x=keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLu activation
x=keras.layers.Dense(1024,activation='relu')(x)
# Add a dropout rate of 0.2
x=keras.layers.Dropout(0.2)(x)
# Add a final a sigmoid layer of classification
x=keras.layers.Dense(1,activation='sigmoid')(x)

model=Model(pre_trained_model.input,x)

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
# Please download the cats_and_dogs_filtered.zip and unzip the zip file
#
# !wget --no-check-certificate \
#         https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
#        -O /tmp/cats_and_dogs_filtered.zip
# local_zip='/tmp/cats_and_dogs_filtered.zip'

# zip_ref=zipfile.ZipFile(local_zip,'r')
# zip_ref.extractall('/tmp')
# zip_ref.close()

    
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile


# Define our example directories and files
base_dir='/tmp/cats_and_dogs_filtered'

train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir,'validation')

train_cats_dir=os.path.join(train_dir,'cats')
train_dogs_dir=os.path.join(train_dir,'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


train_cat_fnames=os.listdir(train_cats_dir)
train_dog_fnames=os.listdir(train_dogs_dir)

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

['cat.952.jpg', 'cat.946.jpg', 'cat.6.jpg', 'cat.749.jpg', 'cat.991.jpg', 'cat.985.jpg', 'cat.775.jpg', 'cat.761.jpg', 'cat.588.jpg', 'cat.239.jpg']
['dog.775.jpg', 'dog.761.jpg', 'dog.991.jpg', 'dog.749.jpg', 'dog.985.jpg', 'dog.952.jpg', 'dog.946.jpg', 'dog.211.jpg', 'dog.577.jpg', 'dog.563.jpg']


In [27]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [28]:
history=model.fit(train_generator,
                 validation_data=validation_generator,
                 steps_per_epoch=100,
                 epochs=20,
                 validation_steps=50,
                 verbose=2
                 )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/20
100/100 - 83s - loss: 0.5026 - accuracy: 0.7575 - val_loss: 0.3355 - val_accuracy: 0.9090
Epoch 2/20
100/100 - 81s - loss: 0.3845 - accuracy: 0.8185 - val_loss: 0.7029 - val_accuracy: 0.8660
Epoch 3/20
100/100 - 86s - loss: 0.3414 - accuracy: 0.8575 - val_loss: 0.3129 - val_accuracy: 0.9370
Epoch 4/20
100/100 - 91s - loss: 0.3345 - accuracy: 0.8600 - val_loss: 0.4001 - val_accuracy: 0.9290
Epoch 5/20
100/100 - 157s - loss: 0.3188 - accuracy: 0.8635


KeyboardInterrupt: 

In [29]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

NameError: name 'history' is not defined